In [1]:
from coiled import Cluster
from dask.distributed import Client

In [ ]:
cluster = Cluster( name="imbalance-join-renre"  
                   , n_workers        = 50
                   , worker_memory    = "64 GiB" 
                   , scheduler_memory = "32 GiB"
                   , package_sync=True
                   ,scheduler_options={"idle_timeout": "2 hour"}  )

In [ ]:
client = Client(cluster)
client

In [5]:
import dask.dataframe as dd

In [6]:
from dask.distributed import performance_report, wait

In [7]:
df1_path = "s3://test-imbalanced-join/df1/"
df2_path = "s3://test-imbalanced-join/df2/"

In [8]:
ddf1 = dd.read_parquet(df1_path)
ddf2 = dd.read_parquet(df2_path)

In [9]:
ddf1

,key,bucket,group1,group2,group3,group4,value
npartitions=7000,,,,,,,
,int64,int64,int64,int64,int64,int64,float64
,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...
,...,...,...,...,...,...,...


In [10]:
ddf2

,df2_group,key
npartitions=1,,
,int64,int64
,...,...


# Step_1

In [11]:
bucket_n_scenario = list(range(1, 40002, 10))
print('bucket_n_scenario [{0}]: {1}'.format(len(bucket_n_scenario), bucket_n_scenario[:50]))

bucket_n_scenario [4001]: [1, 11, 21, 31, 41, 51, 61, 71, 81, 91, 101, 111, 121, 131, 141, 151, 161, 171, 181, 191, 201, 211, 221, 231, 241, 251, 261, 271, 281, 291, 301, 311, 321, 331, 341, 351, 361, 371, 381, 391, 401, 411, 421, 431, 441, 451, 461, 471, 481, 491]


In [12]:
with performance_report(filename='imbalance_synthetic_step01.html'):
    ddf1 = ddf1.set_index('bucket', drop=False, divisions=bucket_n_scenario)
    ddf1 = ddf1.persist()
    wait(ddf1)

# Step_2
 	 	 	  	

In [ ]:
%%time

ddf_bigjoin = ddf1.merge(
      right=ddf2
    , how='inner'
    , on=['key', 'key']
    , suffixes=['_l', '_r']
    )[['df2_group', 'bucket', 'group1', 'group2', 'group3', 'group4', 'value']]


def get_bigjoin_groupby(df):
             
    df = df.groupby(['df2_group', 'bucket', 'group1', 'group2', 'group3', 'group4'], sort=False).agg({'value': 'sum'})
    
    return df


ddf_bigjoin = ddf_bigjoin.map_partitions(get_bigjoin_groupby)


In [15]:
%%time
with performance_report(filename='imbalance_synthetic_step02.html'):
    ddf_bigjoin.to_parquet(
          path="s3://test-imbalanced-join/ouput_join/"
        , engine="pyarrow"                       
        , overwrite=True
        , write_metadata_file=False
        , compression="snappy"
        , write_index=True
        )



CPU times: user 942 ms, sys: 280 ms, total: 1.22 s
Wall time: 4min 4s


In [17]:
len(ddf_bigjoin.value)

63514149896

In [18]:
#len(ddf_bigjoin.value) = 63514149896
63514149896/1e9

63.514149896